В данной статье мы подробно рассмотрим механизм батч-нормализации в нейронных сетях.

Будучи предложенной в 2015 году, батч-нормализация завоевала популярность в сверточных сетях за счет своей эффективности. Было предложено много ее альтернатив для других архитектур. Но батч-нормализация - палка о двух острых концах и имеет много нежелательных свойств (см. раздел [Проблемы батч-нормализации](#problems)):

1. Нарушает независимость между примерами в батче.
2. Делает процесс обучения сильно зависимым от размера батча.
4. Возникает проблема "train-test discrepancy", когда хорошо работающая на трейне модель неожиданно демонстрирует аномальное поведение и низкую точность на тесте.
5. Возникает проблема "медленного старта", когда модель не торопится обучаться, поскольку батч-нормализации требуется "время на разогрев".
3. Возникают неожиданные проблемы при распараллеливании обучения на несколько устройств, вплоть до получения loss=NaN после первого же батча.
6. Большие дополнительные траты вычислительных ресурсов.

И тем не менее, батч-нормализация продолжает активно использоваться, потому что она все-таки работает! Хотя и с предложенным авторами теоретическим обоснованием ее эффективности тоже согласны далеко не все.

<img src="assets/batchnorm.jpg" width="600" align="center">

Содержание:
- [Принцип работы батч-нормализации](#basics)
- [Проблемы батч-нормализации](#problems)

Материал основан на следующих статьях:
- [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167) (Ioffe, Szegedy, 2015)

<a id='basics'></a>
### Необходимые математические понятия

**Среднее и дисперсия**

Пусть мы имеем массив из чисел произвольного размера (например вектор, матрицу или массив более высокой размерности). Давайте повторим понятия выборочного среднего, выборочной дисперсии и среднеквадратичного отклонения:

<style>
td {
  font-size: 14px;
  overflow-wrap: break-word;
}
</style>

| <div style="width:130px">Величина</div> | <div style="width:130px">Описание</div> | <div style="width:150px">Математическая запись</div> | <div style="width:80px">NumPy</div> | <div style="width:100px">TensorFlow</div> | <div style="width:100px">PyTorch</div> |
| :- | :- | :- | :- | :- | :- |
| Выборочное среднее (mean) | Арифметическое среднее всех элементов массива | $\mu=\frac{1}{n}\sum\limits_{i=0}^n X_i$ | `X.mean()` | `tf.math.reduce_mean(X)` | `torch.mean(X)` |
| Выборочная дисперсия (variance) | Арифметическое среднее квадратов отклонений от выборочного среднего | $\sigma^2=\frac{1}{n}\sum\limits_{i=0}^n (X_i-\mu)^2$ | `X.var()` | `tf.math.reduce_variance(X)` | `torch.var(X, unbiased=False)` |
| Среднеквадратичное отклонение (standard deviation) | Квадратный корень из выборочной дисперсии | $\sigma=\sqrt{\frac{1}{n}\sum\limits_{i=0}^n (X_i-\mu)}$ | `X.std()` | `tf.math.reduce_std(X)` | `torch.std(X, unbiased=False)` |

*Примечание 1.* У многомерного массива элемент имеет несколько индексов, но для простоты математической записи будем считать, что все элементы массива можно перебрать одним индексом. В NumPy это эквивалентно перебору массива `array.flat` или `array.reshape(-1)`.

*Примечание 2.* Есть и другой вариант дисперсии, называемый исправленной дисперсией (поправка Бесселя). В этом варианте $1/n$ заменяется на $1/(n-1)$. PyTorch по умолчанию считает дисперсию с этой поправкой, а NumPy и TensorFlow по умолчанию считают без поправки. Чтобы результаты совпали, в вызовы PyTorch следует добавить `unbiased=False`. Все то же самое касается и среднеквадратичного отклонения.

*Примечание 3.* Для случайной величины среднее значение называется *математическим ожиданием*. Среднее и дисперсия называются, соответственно, *первым и вторым центральными моментами* случайной величины или выборки. В TensorFlow есть также функция `tf.nn.moments` для расчета одновременно среднего и дисперсии массива.

**Стандартизация (нормализация) массива**

Стандартизацией называется процесс, когда мы отнимаем от массива его среднее и делим на среднеквадратичное отклонение:

$X_{st} = \cfrac{X - \mu}{\sigma}$

Полученный в результате массив будет обладать нулевым средним и единичной дисперсией. Замечено, что если в задаче машинного обучения каждый признак подвергнуть такому преобразованию, то ML-модели как правило начинают эффективнее обучаться (см. например `sklearn.preprocessing.StandardScaler`).

Конечно, стандартизация не имеет смысла для массива из одного элемента, так как дисперсия будет равна нулю (или будет неопределенной с поправкой Бесселя). Кроме того, стандартизация не сработает для массива, все элементы которого равны, потому что в этом случае дисперсия также будет равна нулю. Для избавления от этой проблемы в формулу иногда добавляют коэффициент "численной стабильности" $\epsilon$, который очень близок к нулю, например равен $10^{-5}$. В этом случае знаменатель никогда не обратится в ноль:

$X_{st} = \cfrac{X - \mu}{\sqrt{\sigma^2 + \epsilon}}$

Иногда стандартизацию называют также ***нормализацией***. В целом "нормализация" - это широкое понятие, которое может иметь разные значения. В батч-нормализации, о которой далее пойдет речь, под "нормализацией" понимается именно стандартизация. Поэтому далее будем употреблять термин "нормализация" вместо "стандартизации".

**Нормализация массива по отдельным осям**

Пусть мы имеем numpy-массив `array` с $N$ осями и хотим, чтобы для любого `i` подмассив `array[..., i]` был нормализован, то есть имел нулевое среднее и единичную дисперсию. Если мы нормализуем весь массив целиком, то не добьемся желаемого свойства. Вместо этого нам нужно нормализовывть отдельно по каждому индексу:

In [23]:
import numpy as np
array = np.random.rand(8, 16, 32, 64)

for i in range(array.shape[-1]):
    mean = array[..., i].mean()
    std = array[..., i].std()
    array[..., i] = (array[..., i] - mean) / std

Используя возможности numpy, можно сделать это эффективнее:

In [22]:
axes = tuple(range(array.ndim))[:-1]
means = array.mean(axis=axes, keepdims=True)
stds = array.std(axis=axes, keepdims=True)
array = (array - means) / stds

Если исходный массив имел размер `(8, 16, 32, 64)`, то массивы `means` и `stds` в данном коде будут иметь размер `(1, 1, 1, 64)`, то есть содержать по 64 числа. 
Описанное выше преобразование является примером нормализации по отдельным осям. Именно такая операция применяется в батч-нормализации, о которой пойдет речь далее.

**Экспоненциальное сглаживание**

Это еще одно понятие, которое нам потребуется. Пусть мы имеем временной ряд из чисел $\{x_t\}_{t=0}^N$. Этот ряд может быть "зашумлен", и нам хотелось бы избавиться от шума. Для этого мы можем применить к массиву экспоненциальное сглаживание с заранее выбранным коэффициентом $\alpha$. Сглаженный ряд $\{s_t\}_{t=0}^N$ получается из исходного ряда $\{x_t\}_{t=0}^N$ по следующей формуле:

$s_0 = 0$ (или другое начальное значение)

$s_t = \alpha s_{t-1} + (1-\alpha) x_t$

Если в данной формуле выразить $s_{t-1}$ через $s_{t-2}$, затем выразить $s_{t-2}$ через $s_{t-3}$ и так далее, то получим, что применение экспоненциального сглаживания эквивалентно свертке исходного ряда с экспоненциальной функцией.

На примере ниже исходный ряд отмечен синим цветом, сглаженный - оранжевым цветом:

<img src="assets/smooth.jpg" width="500" align="center">

### Батч-нормализация в полносвязных сетях

Рассмотрим полносвязную нейронную сеть, которая принимает на вход набор признаков. Нейроны первого слоя выдают числа - эти числа играют роль "признаков", которые обрабатывает следующий слой, и так далее. Если  мы подаем на вход сети случайный пример, то выход каждого нейрона можно считать случайной величиной, распределение вероятностей которой (в том числе мат. ожидание и дисперсия) зависит от весов сети и датасета. Авторы батч-нормализации стремятся к тому, чтобы по каждому нейрону каждого слоя мат. ожидание было как можно ближе к нулю, а дисперсия была как можно ближе к единице.

*Примечание.* Почему авторы батч-нормализации ставят такую цель и что об этом говорят авторы более поздних работ - мы рассмотрим в теоретической части данного обзора.

Для достижения этой цели предлагается добавлять между слоями сети дополнительные слои батч-нормализации (BN). Эти слои осуществляют нормализацию данных, однако они делают это по-разному в режимах обучения и инференса.

Пусть предыдущий слой имел $N$ нейронов, а размер батча равен $B$. Тогда слой батч-нормализации принимает массив данных `T` размера $(B, N)$ и возвращает массив.

**В режиме обучения** слой BN нормализует массив таким образом, что для любого нейрона $i$ массив выданных на данном батче значений `T[:, i]` имеет нулевое среднее и единичную дисперсию. Например, если размер батча равен 4, и 1-й нейрон выдал значения `[0.0, 0.1, 0.2, 0.3]`, то после слоя батч-нормализации эти значения превратятся в `[-1.34, -0.45, 0.45, 1.34]`:

In [30]:
import numpy as np
X = np.array([0.0, 0.1, 0.2, 0.3])
epsilon = 1e-5
(X - X.mean()) / np.sqrt(X.var() + epsilon)

array([-1.34110445, -0.44703482,  0.44703482,  1.34110445])

Отсюда можно сделать два вывода:

1. Размер батча должен быть больше единицы, так как нормализация массива из одного элемента не имеет смысла.
2. Примеры в батче теперь будут влиять друг на друга, так как выход каждого нейрона зависит от среднего и дисперсии по данному нейрону, а они, в свою очередь, зависят от всех остальных примеров в батче.

Можно также сказать, что среднее и дисперсия по батчу - это аппроксимация мат. ожидания и дисперсии по всему порождающему распределению (читай *датасету*), причем чем больше размер батча - тем точнее аппроксимация.

**В режиме инференса** такой подход слишком ненадежен, поскольку:
1. Нам не хотелось бы, чтобы примеры в батче влияли друг на друга.
2. Модель должна уметь обрабатывать батчи из одного примера.

Несоблюдение этих правил сильно понизит практическую ценность моделей, обученных с батч-нормализацией. Поэтому для режима инференса нужно искать другой подход.

Авторы предлагают следующее решение: брать коэффициенты нормализации из статистики по обучающим данным. После обучения у нас остается статистика среднего и дисперсии *по каждому нейрону*. Если было $S$ шагов обучения, то *для каждого нейрона* осталось $S$ значений среднего $\{\mu_t\}_{t=0}^S$ и $S$ значений дисперсии $\{\sigma_t^2\}_{t=0}^S$ - по одному для каждого обучающего батча. Мы можем использовать эти значения. Среднее и дисперсия могли меняться в процессе обучения, поэтому нас интересуют значения ближе к концу обучения. Можно взять значения на последнем батче - но это слишком ненадежно, ведь в батче не очень много примеров. В качестве компромисса мы применяем к массивам $\{\mu_t\}_{t=0}^S$ и $\{\sigma_t^2\}_{t=0}^S$ экспоненциальное сглаживание.

Таким образом, для каждого нейрона слой BN хранит два числовых параметра: `running_mean` и `running_variance`. Эти параметры обновляются на каждом шаге обучения, но не градиентным спуском, а по формуле экспоненциального сглаживания, с использованием $\mu$ и $\sigma^2$, расчитываемых на каждом обучающем батче. Параметр сглаживания $\alpha$, называемый моментом, является гиперпараметром слоя BN.

*Примечание*. Здесь полная аналогия с тем, как работает градиентный спуск с моментом. В нем величина градиента также подвергается экспоненциальному сглаживанию. В адаптивном градиентном спуске (Adagrad, Adam) таким же образом сглаживается второй момент градиента.

Если принцип обновления `running_mean` и `running_variance` остался не до конца понятен, то далее будет показана программная реализация батч-нормализации.

**Батч нормализация до или после функции активации?**

И действительно, у нас есть два варианта: либо ставить BN после линейного преобразования и до функции активации слоя ("нелинейности"), либо после функции активации. Авторы в оригинальной статье предлагают ставить BN до нелинейности, однако не дают этому внятного объяснения. Вспоследствии было выяснено, что BN после нелинейности дает во многих случаях лучшие результаты, и даже один из авторов работы (про второго неизвестно) впоследствии стал делать именно так:

> I can guarantee that recent code written by Christian [Szegedy] applies relu before BN. It is still occasionally a topic of debate, though. (François Chollet, 2016, [link](https://github.com/keras-team/keras/issues/1802#issuecomment-187966878))

**Дополнительное аффинное преобразование**

Рассмотрим случай, когда BN применяется до нелинейности (как в оригинальной работе). Тогда значения, поступающие в функцию активации, имеют нулевое среднее и единичную дисперсию (расчитанные по обучающему батчу). Вероятно, больше половины значений окажется в интервале от -1 до +1. Если функцией активации является сигмоида или tanh, то на этом интервале функция почти линейна. Но основной смысл функции активации - именно в ее нелинейности! Конечно можно просто умножить аргумент на 2 либо использовать ReLU, но авторы предлагают пойти другим путем, более универсальным.

Для каждого нейрона вводится два обучаемых параметра: scale ($\gamma$) и bias ($\beta$). Выходные значения слоя BN дополнительно преобразуются по формуле:

$y = \gamma x + \beta$

Параметры $\gamma$ и $\beta$ являются обучаемыми, то есть по ним расчитываются производные и они обновляются градиентным спуском по тому же принципу, что и остальные веса сети. Параметры $\gamma$ и $\beta$ являются независимыми для каждого нейрона, то есть если полносвязный слой имеет $N$ нейронов, то стоящий после него (до или после функции активации) слой BN имеет $4N$ параметров:

1. $2N$ обучаемых градиентным спуском параметров ($\gamma$, $\beta$).
1. $2N$ не обучаемых градиентным спуском параметров (`running_mean`, `running_variance`).

<img src="assets/batchnorm2.jpg" width="700" align="center">

**Следствия**

Можно сразу отметить две важные детали насчет слоев BN:

*Следствие 1*. Если слой BN ставится до функции активации, то в слое можно убрать bias'ы, поскольку их влияние полностью нивелируется идущим сразу после них слоем BN. Не важно чему равны bias'ы полносвязного слоя - значения, выдаваемые слоем BN, от этого не изменятся. Это можно проверить по формулам, отдельно для обучения и инференса.

*Следствие 2*. Если слой BN ставится после функции активации, то аффинное преобразование (параметры $\gamma$, $\beta$) не добавляют выразительной способности сети, так как сразу после них идет полносвязный слой, а два линейных преобразования подряд эквивалентны одному. Но с другой стороны, даже если параметры $\gamma$ и $\beta$ не влияют на выразительную способность, то они все же могут повлиять на результат оптимизации (обучения). Так что вопрос их влияния неоднозначен.

### Батч-нормализация в сверточных сетях

Сверточные сети могут обрабатывать данные с разным количеством осей (1D, 2D, 3D, ...), поэтому для примера рассмотрим сверточные сети, которые выполняют свертки по двум осям и работают с изображениями. Аналогом нейрона здесь является фильтр, поэтому нормализация выполняется *не по каждому нейрону в отдельности, а по каждому фильтру в отдельности*. Отличие только в размерности данных.

Слой BN принимает на вход массив с четырьмя осями. В зависимости от кокретной библиотеки (TensorFlow, PyTorch и др.) порядок этих осей может меняться. Рассмотрим на примере PyTorch:

- **Ось 0** - номер примера в батче $N$
- **Ось 1** - номер фильтра предыдущего сверточного слоя $C$
- **Ось 2** - позиция по высоте изображения $H$
- **Ось 3** - позиция по ширине изображения $W$

*(для сравнения, в TensorFlow по умолчанию используется порядок осей "channel_last", номер фильтра является последней осью)*

Зафиксировав номер фильтра, мы получим трехмерный массив размером ($N$, $H$, $W$). Как раз данный массив и нормализуется, то есть после преобразования среднее по этому массиву будет равно 0, а дисперсия будет равна 1. На примере numpy это означает следующую операцию:

In [ ]:
means = inputs.mean(axis=(0, 2, 3), keepdims=True)
variances = inputs.var(axis=(0, 2, 3), keepdims=True)
outputs = (inputs - means) / np.sqrt(variances + epsilon)

Слой BN точно так же имеет $4N$ параметров, где $N$ - количество фильтров в предыдущем сверточном слое.

### Пример программной реализации

Пример на PyTorch взят [из данного источника](http://d2l.ai/chapter_convolutional-modern/batch-norm.html?highlight=batchnorm2d), там же есть пример на Keras.

In [37]:
import torch
from torch import nn

def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # Use `is_grad_enabled` to determine whether the current mode is training
    # mode or prediction mode
    if not torch.is_grad_enabled():
        # If it is prediction mode, directly use the mean and variance
        # obtained by moving average
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # When using a fully-connected layer, calculate the mean and
            # variance on the feature dimension
            mean = X.mean(dim=0)
            var = ((X - mean)**2).mean(dim=0)
        else:
            # When using a two-dimensional convolutional layer, calculate the
            # mean and variance on the channel dimension (axis=1). Here we
            # need to maintain the shape of `X`, so that the broadcasting
            # operation can be carried out later
            mean = X.mean(dim=(0, 2, 3), keepdim=True)
            var = ((X - mean)**2).mean(dim=(0, 2, 3), keepdim=True)
        # In training mode, the current mean and variance are used for the
        # standardization
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # Update the mean and variance using moving average
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # Scale and shift
    return Y, moving_mean.data, moving_var.data

class BatchNorm(nn.Module):
    # `num_features`: the number of outputs for a fully-connected layer
    # or the number of output channels for a convolutional layer. `num_dims`:
    # 2 for a fully-connected layer and 4 for a convolutional layer
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # The scale parameter and the shift parameter (model parameters) are
        # initialized to 1 and 0, respectively
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # The variables that are not model parameters are initialized to 0 and 1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        # If `X` is not on the main memory, copy `moving_mean` and
        # `moving_var` to the device where `X` is located
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # Save the updated `moving_mean` and `moving_var`
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean, self.moving_var,
            eps=1e-5, momentum=0.9)
        return Y

Давайте рассмотрим этот пример подробнее.

**Общая структура кода**

Как принято в PyTorch, мы создаем модуль `BatchNorm`, который наследуем от `torch.nn.Module`, и в нем ссылаемся на stateless-функцию `batch_norm`. В PyTorch такие функции лежат в модуле `torch.nn.functional`, но здесь мы пишем свою функцию.

Режим работы слоя (обучение или инференс) мы узнаем с помощью переменной `torch.is_grad_enabled()`. В качестве альтернативы можно было узнать режим работы в методе `forward` и написать две отдельные stateless-функции для обучения и для инференса.

**Инициализация слоя**

В слое `BatchNorm` инициализируется 4 параметра: `gamma`, `beta`, `moving_mean`, `moving_var`. При этом `moving_mean` и `beta` инициализируется нулями, а `moving_var` и `gamma` единицами. Все эти 4 массива параметров имеют то же количество осей, что и входной массив данных, при этом размерность всех осей, кроме номера фильтра/нейрона, равна единице.

**В режиме обучения** мы используем статистики по батчу, для этого мы считаем среднее и дисперсию по всем осям, кроме номера фильтра/нейрона:

`mean = X.mean(dim=0)` - полносвязная сеть
<br>`mean = X.mean(dim=(0, 2, 3), keepdim=True)` - сверточная сеть

Мы также обновляем `moving_mean` и `moving_var` по правилу экспоненциального сглаживания, используя `momentum`:

`moving_mean = momentum * moving_mean + (1.0 - momentum) * mean`
<br>`moving_var = momentum * moving_var + (1.0 - momentum) * var`

Чем ближе `momentum` к единице, тем более сильное сглаживания мы применяем (но при этом растет и запаздывание сглаженного графика по сравнению с исходным).

**В режиме инференса** мы используем `moving_mean` и `moving_var` в качестве среднего и дисперсии.

В обоих режимах мы также применяем аффинное преобразование:

`Y = gamma * X_hat + beta`

### Как батч-нормализация работает на практике?

Давайте обратимся к разделу "эксперименты" в оригинальной статье. Авторы применили батч-нормализацию к сети [Inception (GoogLeNet)]($Going Deeper with Convolutions$) на задаче классификации ImageNet.

<img src="assets/batchnorm3.jpg" width="500" align="center">

В таблице показана зависимость точности на валидации от номера шага оптимизации для нескольких сетей:

*Incepion* - исходная сеть Inception без изменений. Использовался cтартовый learning rate $1.5 * 10^{-3}$, который затем экспоненциально затухал.

*BN-Baseline* - сеть Inception, в которой перед каждой нелинейностью ReLU была добавлена батч-нормализация.

*BN-x5* - BN-Baseline со следующими изменениями:

- стартовый learning rate повышен в 5 раз
- скорость затухания LR повышена в 6 раз
- удален dropout
- L2-регуляризация весов уменьшена в 5 раз
- удален Local Response Normalization
- уменьшены аугментации

Для сравнения, исходная сеть расходилась с таким learning rate, то есть батч-нормализация позволяет обучать сеть с более высоким learning rate. Dropout авторы удалили потому, что обнаружили: его удаление ускоряет обучение без усиления переобучения. В исходной же сети удаление dropout усиливало переобучение. Поэтому авторы предполагают, что BN обладает регуляризующим эффектом и может частично или полностью заменить собой dropout.

*BN-x30* - BN-x5, в которй стартовый learning rate был увеличен не в 5, а в 30 раз по сравнению с Baseline. Даже такая сеть хорошо обучается, а значит BN позволяет очень сильно повышать learning rate без расходимости сети.

*BN-x5-Sigmoid* - BN-x5, в которой ReLU был заменен на сигмоиду. Для сравнения, исходная сеть (без BN) с сигмоидой вообще не могла обучиться (точность оставалась на уровне случайного угадывания).

Таким образом, батч-нормализация повышает точность на валидации, позволяет обучать с намного более высоким learning rate, позволяет обучать глубокие сети с сигмоидой и возможно обладает регуляризующим эффектов. Причины описанных явлений будут еще обсуждаться в следующих разделах.

**Мои эксперименты с разными функциями активации**

Хотел бы также упомянуть о собственных некогда проведенных экспериментах, где я сравнивал на практике различные функции активации с батч-нормализацией и без нее. Я составил список из около 25 разных функций активации, и для каждой функции активации обучал по 10 нейросетей на наборе Extended MNIST, нормированном от 0 до 1, используя сверточную сеть с фильтрами 3x3 следующей конфигурации (в скобках - кол-во фильтров/нейронов):

```
Conv2D(8)
Conv2D(16)
MaxPooling2D(2x2)
Conv2D(16)
Conv2D(32)
MaxPooling2D(2x2)
Dense(500)
Dense(10)
```

Выбранная функция активации применялась во всех сверточных слоях и скрытом полносвязном слое. Размер батча был взят равным 200, оптимизатор adam, loss - категориальная кроссэнтропия. Результаты эксперимента показаны на 9 изображениях ниже.